#1-Validação do schema de cada tabela, ccheck e tratamento de valores null.

In [0]:
%python
from pyspark.sql import functions as F
from pyspark.sql.types import *

silver_catalog = "mvp_datasus.silver"

tables = [r["tableName"] for r in spark.sql(f"SHOW TABLES IN {silver_catalog}").collect()]

summary = []

for name in tables:
    table_full = f"{silver_catalog}.`{name}`"
    print("\n" + "="*80)
    print(f"Tabela: {name}")

    df = spark.table(table_full)
    total_rows = df.count()
    print(f" Linhas: {total_rows}")

    schema = df.schema
    cols = [f for f in schema.fields]

    # contar nulls antes (somatório por coluna)
    nulls_before = {}
    for f in cols:
        c = f.name
        dtype = f.dataType
        if isinstance(dtype, (FloatType, DoubleType)):
            n = df.filter(F.isnan(F.col(c)) | F.col(c).isNull()).count()
        elif isinstance(dtype, StringType):
            n = df.filter(F.col(c).isNull() | (F.col(c) == "")).count()
        else:
            n = df.filter(F.col(c).isNull()).count()
        nulls_before[c] = n

    total_nulls_before = sum(nulls_before.values())
    print(f" Total de células nulas (antes): {total_nulls_before}")

    # expressões para correção por coluna
    exprs = []
    skipped_cols = []
    for f in cols:
        c = f.name
        dtype = f.dataType

        if isinstance(dtype, (ByteType, ShortType, IntegerType, LongType, FloatType, DoubleType, DecimalType)):
            # numéricos: substituir isnan / null por 0, preservando tipo
            lit0 = F.lit(0).cast(dtype)
            if isinstance(dtype, (FloatType, DoubleType)):
                corrected = F.when(F.isnan(F.col(c)) | F.col(c).isNull(), lit0).otherwise(F.col(c)).alias(c)
            else:
                corrected = F.when(F.col(c).isNull(), lit0).otherwise(F.col(c)).alias(c)
            exprs.append(corrected)

        elif isinstance(dtype, StringType):
            # string: null or empty -> '0'
            corrected = F.when(F.col(c).isNull() | (F.col(c) == ""), F.lit("0")).otherwise(F.col(c)).alias(c)
            exprs.append(corrected)

        elif isinstance(dtype, BooleanType):
            # boolean: null -> False
            corrected = F.when(F.col(c).isNull(), F.lit(False)).otherwise(F.col(c)).alias(c)
            exprs.append(corrected)

        elif isinstance(dtype, (DateType, TimestampType)) or isinstance(dtype, (ArrayType, StructType, MapType)):
            # não altera tipos de data/timestamp
            skipped_cols.append(c)
            exprs.append(F.col(c))
        else:
            
            skipped_cols.append(c)
            exprs.append(F.col(c))

    # aplicar correções
    df_fixed = df.select(*exprs)

    # contar nulls depois (somatório por coluna)
    nulls_after = {}
    for f in df_fixed.schema.fields:
        c = f.name
        dt = f.dataType
        if isinstance(dt, (FloatType, DoubleType)):
            n = df_fixed.filter(F.isnan(F.col(c)) | F.col(c).isNull()).count()
        elif isinstance(dt, StringType):
            n = df_fixed.filter(F.col(c).isNull() | (F.col(c) == "")).count()
        else:
            n = df_fixed.filter(F.col(c).isNull()).count()
        nulls_after[c] = n

    total_nulls_after = sum(nulls_after.values())
    corrected_count = total_nulls_before - total_nulls_after

    # sobrescrever a tabela no metastore
    try:
        (df_fixed.write
            .format("delta")
            .mode("overwrite")
            .option("overwriteSchema", "true")
            .saveAsTable(table_full))
        written = True
    except Exception as e:
        print(f" Erro sobrescrevendo tabela {table_full}: {e}")
        written = False

    # resumo por tabela
    print(f" Total de células nulas (depois): {total_nulls_after}")
    print(f" Corrigidas: {corrected_count}")
    if skipped_cols:
        print(f" Colunas puladas (date/timestamp/complex): {skipped_cols}")
    else:
        print(" Todas as colunas primitivas foram tratadas.")

    summary.append({
        "table": name,
        "rows": total_rows,
        "nulls_before": total_nulls_before,
        "nulls_after": total_nulls_after,
        "corrected": int(corrected_count),
        "skipped_columns": skipped_cols,
        "written": written
    })

    # mostrar 5 primeiras linhas da tabela atualizada
    print(" Display das 5 primeiras linhas (após correção):")
    display(spark.table(table_full).limit(5))

# resumo final
print("\n" + "="*80)
print("RESUMO GERAL:")
for s in summary:
    print(f" - {s['table']}: rows={s['rows']}, nulls_before={s['nulls_before']}, nulls_after={s['nulls_after']}, corrected={s['corrected']}, skipped={len(s['skipped_columns'])}, written={s['written']}")


Tabela: obitos_geral
 Linhas: 27
 Total de células nulas (antes): 0
 Total de células nulas (depois): 0
 Corrigidas: 0
 Todas as colunas primitivas foram tratadas.
 Display das 5 primeiras linhas (após correção):


cod_ibge,unidade_federacao,2020,2021,2022,2023,2024,2025
11,Rondônia,4135,5725,3850,3983,3981,2706
12,Acre,1907,2325,1623,1753,1811,1109
13,Amazonas,9914,11544,7366,6913,7200,5332
14,Roraima,1696,2264,1507,1492,1084,436
15,Pará,15224,18017,14246,14500,14958,10709



Tabela: obitos_hiv_faixa_0_10
 Linhas: 16
 Total de células nulas (antes): 0
 Total de células nulas (depois): 0
 Corrigidas: 0
 Todas as colunas primitivas foram tratadas.
 Display das 5 primeiras linhas (após correção):


cod_ibge,unidade_federacao,2020,2021,2022,2023,2024,2025
13,Amazonas,0,1,1,1,1,0
15,Pará,1,1,1,1,0,0
21,Maranhão,0,0,1,0,1,0
22,Piauí,0,0,1,0,0,0
25,Paraíba,1,0,0,0,0,0



Tabela: obitos_hiv_faixa_0_10_fem
 Linhas: 11
 Total de células nulas (antes): 0
 Total de células nulas (depois): 0
 Corrigidas: 0
 Todas as colunas primitivas foram tratadas.
 Display das 5 primeiras linhas (após correção):


...


Tabela: obitos_hiv_geral
 Linhas: 27
 Total de células nulas (antes): 0
 Total de células nulas (depois): 0
 Corrigidas: 0
 Todas as colunas primitivas foram tratadas.
 Display das 5 primeiras linhas (após correção):


cod_ibge,unidade_federacao,2020,2021,2022,2023,2024,2025
11,Rondônia,24,61,46,64,59,45
12,Acre,8,4,4,8,19,4
13,Amazonas,188,226,210,194,224,139
14,Roraima,27,32,24,28,17,7
15,Pará,119,116,77,77,78,57



Tabela: obitos_hiv_masc
 Linhas: 27
 Total de células nulas (antes): 0
 Total de células nulas (depois): 0
 Corrigidas: 0
 Todas as colunas primitivas foram tratadas.
 Display das 5 primeiras linhas (após correção):


cod_ibge,unidade_federacao,2020,2021,2022,2023,2024,2025
11,Rondônia,10,49,30,51,46,30
12,Acre,6,3,4,7,15,4
13,Amazonas,138,169,162,152,161,107
14,Roraima,23,22,21,20,11,5
15,Pará,81,78,51,54,53,36



RESUMO GERAL:
 - obitos_geral: rows=27, nulls_before=0, nulls_after=0, corrected=0, skipped=0, written=True
 - obitos_hiv_faixa_0_10: rows=16, nulls_before=0, nulls_after=0, corrected=0, skipped=0, written=True
 - obitos_hiv_faixa_0_10_fem: rows=11, nulls_before=0, nulls_after=0, corrected=0, skipped=0, written=True
 - obitos_hiv_faixa_0_10_masc: rows=8, nulls_before=0, nulls_after=0, corrected=0, skipped=0, written=True
 - obitos_hiv_faixa_10_25: rows=27, nulls_before=0, nulls_after=0, corrected=0, skipped=0, written=True
 - obitos_hiv_faixa_10_25_fem: rows=24, nulls_before=0, nulls_after=0, corrected=0, skipped=0, written=True
 - obitos_hiv_faixa_10_25_masc: rows=27, nulls_before=0, nulls_after=0, corrected=0, skipped=0, written=True
 - obitos_hiv_faixa_25_40: rows=27, nulls_before=0, nulls_after=0, corrected=0, skipped=0, written=True
 - obitos_hiv_faixa_25_40_fem: rows=27, nulls_before=0, nulls_after=0, corrected=0, skipped=0, written=True
 - obitos_hiv_faixa_25_40_masc: rows=27, 